# bounding box坐标定义：两种方式

In [2]:
# 一种是x1,y1,x2,y2即左上右下两个矩形框坐标点，这里有个默认的信息其实是x2>x1，y2>y1
# 二种是cx,cy,w,h即矩形中心点坐标、宽、高
# 下面定义两种坐标表示方式的相互转换函数
import torch
def xyTocxcy(xy):
    cxcy = torch.cat(((xy[:, 2:] + xy[:,:2])/2, xy[:, 2:] - xy[:,:2]) , 1)
    return cxcy

def cxcyToxy(cxcy):
    xy = torch.cat((cxcy[:,:2] - (cxcy[:, 2:]/2), cxcy[:,:2] + (cxcy[:, 2:]/2)), 1)
    return xy

# 简单测试一下代码
x = torch.tensor([[1,2,3,4], [5,6,7,8]])
print(x)
y = xyTocxcy(x)
print(y)
print(cxcyToxy(y))

tensor([[1, 2, 3, 4],
        [5, 6, 7, 8]])
tensor([[2., 3., 2., 2.],
        [6., 7., 2., 2.]])
tensor([[1., 2., 3., 4.],
        [5., 6., 7., 8.]])


# 交并比（IoU)

In [ ]:
# 交并比其实很好理解就是两个框相交的面积除以两个框取并集的面积，衡量比例
def find_intersection(set_1, set_2):
    """ 
    Find the intersection of every box combination between two sets of boxes that are in boundary coordinates.

    :param set_1: set 1, a tensor of dimensions (n1, 4)                                                                                                           
    :param set_2: set 2, a tensor of dimensions (n2, 4)
    :return: intersection of each of the boxes in set 1 with respect to each of the boxes in set 2, a tensor of dimensions (n1, n2)
    """

    # PyTorch auto-broadcasts singleton dimensions
    lower_bounds = torch.max(set_1[:, :2].unsqueeze(1), set_2[:, :2].unsqueeze(0))  # (n1, n2, 2)
    upper_bounds = torch.min(set_1[:, 2:].unsqueeze(1), set_2[:, 2:].unsqueeze(0))  # (n1, n2, 2)
    intersection_dims = torch.clamp(upper_bounds - lower_bounds, min=0)  # (n1, n2, 2)
    # clamp函数相当于minmax，定义为min(min(x, min), max)，意味着要规定下限和上限，默认无上限和下限
    return intersection_dims[:, :, 0] * intersection_dims[:, :, 1]  # (n1, n2)


def find_jaccard_overlap(set_1, set_2):
    """ 
    Find the Jaccard Overlap (IoU) of every box combination between two sets of boxes that are in boundary coordinates.

    :param set_1: set 1, a tensor of dimensions (n1, 4)
    :param set_2: set 2, a tensor of dimensions (n2, 4)
    :return: Jaccard Overlap of each of the boxes in set 1 with respect to each of the boxes in set 2, a tensor of dimensions (n1, n2)
    """

    # Find intersections
    intersection = find_intersection(set_1, set_2)  # (n1, n2)

    # Find areas of each box in both sets
    areas_set_1 = (set_1[:, 2] - set_1[:, 0]) * (set_1[:, 3] - set_1[:, 1])  # (n1)
    areas_set_2 = (set_2[:, 2] - set_2[:, 0]) * (set_2[:, 3] - set_2[:, 1])  # (n2)

    # Find the union
    # PyTorch auto-broadcasts singleton dimensions
    union = areas_set_1.unsqueeze(1) + areas_set_2.unsqueeze(0) - intersection  # (n1, n2)

    return intersection / union  # (n1, n2)
